In [5]:
import ctypes
import locale

import time
# 일시 정지 후 실행할 코드
print("5초 후 실행됩니다.")
# 5초 동안 프로그램 실행을 일시 정지합니다.
time.sleep(5)

user32 = ctypes.WinDLL('user32', use_last_error=True)
VK_HANGUL = 0x15
# Get the current keyboard layout
klid = user32.GetKeyboardLayout(0)
# Use GetKeyboardState to reliably check the toggle state of the Hangul key
key_states = (ctypes.c_ubyte * 256)()
user32.GetKeyboardState(key_states)
state = key_states[VK_HANGUL]
language_name = locale.windows_locale[klid & 0xFFFF]
korean_mode = 'Korean' if state & 1 else 'English'
print(f'Current input language: {language_name} ({korean_mode})')


5초 후 실행됩니다.
Current input language: ko_KR (Korean)


In [ ]:
import ctypes
from ctypes import wintypes



# Load necessary DLLs
user32 = ctypes.windll.user32
imm32 = ctypes.windll.imm32

# Constants
IME_CMODE_HANGUL = 0x0001
LANG_KOREAN = 0x0412

def get_keyboard_mode():
    # Get current keyboard layout
    hkl = user32.GetKeyboardLayout(0)
    lang_id = hkl & 0xFFFF  # Extract the language ID from the HKL

    if lang_id != LANG_KOREAN:
        return "English"  # Not a Korean keyboard layout

    # Get the foreground window (to check IME status)
    hwnd = user32.GetForegroundWindow()
    print(hwnd)
    # Get the input context for the current window
    himc = imm32.ImmGetContext(hwnd)
    if not himc:
        print(himc)
        return "Korean (En)"  # No IME context available, assume English

    # Get conversion status from IME
    conv_mode = wintypes.DWORD()
    sent_mode = wintypes.DWORD()
    success = imm32.ImmGetConversionStatus(himc, ctypes.byref(conv_mode), ctypes.byref(sent_mode))

    # Release the input context
    imm32.ImmReleaseContext(hwnd, himc)

    if not success:
        return "Korean (En)"  # Failed to get conversion status
    
    # Check if Hangul mode is active
    if conv_mode.value & IME_CMODE_HANGUL:
        return "Korean (Ko)"
    else:
        return "Korean (En)"

# Example usage
if __name__ == "__main__":
    mode = get_keyboard_mode()
    print(f"Current keyboard mode: {mode}")

5초 후 실행됩니다.
592690
0
Current keyboard mode: Korean (En)


In [ ]:
import ctypes
from ctypes import wintypes

import time
# 일시 정지 후 실행할 코드
print("5초 후 실행됩니다.")
# 5초 동안 프로그램 실행을 일시 정지합니다.
time.sleep(5)


user32 = ctypes.windll.user32

# Constants
VK_HANGUL = 0x15  # Virtual key code for Hangul toggle
LANG_KOREAN = 0x0412

def get_keyboard_mode():
    # Get current keyboard layout
    hkl = user32.GetKeyboardLayout(0)
    lang_id = hkl & 0xFFFF

    if lang_id != LANG_KOREAN:
        return "English"  # Not a Korean keyboard layout

    # Check if the Hangul key is toggled (IME in Hangul mode)
    hangul_state = user32.GetKeyState(VK_HANGUL)
    if hangul_state & 0x0001:  # Low-order bit indicates toggle state
        return "Korean (Ko)"
    else:
        return "Korean (En)"

# Example usage
if __name__ == "__main__":
    mode = get_keyboard_mode()
    print(f"Current keyboard mode: {mode}")

5초 후 실행됩니다.
Current keyboard mode: Korean (Ko)


In [ ]:
import ctypes
from ctypes import wintypes

user32 = ctypes.windll.user32

# Constants
VK_A = 0x41  # Virtual key code for 'A'
LANG_KOREAN = 0x0412

def get_keyboard_mode():
    # Get the foreground window's thread and keyboard layout
    hwnd = user32.GetForegroundWindow()
    pid = wintypes.DWORD()
    thread_id = user32.GetWindowThreadProcessId(hwnd, ctypes.byref(pid))
    hkl = user32.GetKeyboardLayout(thread_id)
    
    # Check if the layout is Korean
    lang_id = hkl & 0xFFFF
    if lang_id != LANG_KOREAN:
        return "English"
    
    # Get current keyboard state (ignore modifiers like Shift/CapsLock)
    keyboard_state = (ctypes.c_ubyte * 256)()
    user32.GetKeyboardState(keyboard_state)
    # Reset modifier states to avoid interference
    keyboard_state[user32.VK_SHIFT] = 0
    keyboard_state[user32.VK_CONTROL] = 0
    keyboard_state[user32.VK_MENU] = 0  # Alt key
    
    # Buffer to hold the translated Unicode character(s)
    char_buffer = ctypes.create_unicode_buffer(16)
    
    # Translate the 'A' key press to Unicode
    chars_written = user32.ToUnicodeEx(
        VK_A, 0, 
        keyboard_state, 
        char_buffer, 16, 
        0,  # Flags
        hkl  # Keyboard layout
    )
    
    if chars_written > 0:
        # Check if the output is a Hangul character (Unicode range AC00–D7A3)
        char_code = ord(char_buffer[0])
        if 0xAC00 <= char_code <= 0xD7A3:
            return "Korean (Ko)"
        else:
            return "Korean (En)"
    else:
        return "Korean (En)"  # Fallback

if __name__ == "__main__":
    print(f"Current keyboard mode: {get_keyboard_mode()}")

AttributeError: function 'VK_SHIFT' not found

In [ ]:
import ctypes
from ctypes import wintypes

import time
# 일시 정지 후 실행할 코드
print("5초 후 실행됩니다.")
# 5초 동안 프로그램 실행을 일시 정지합니다.
time.sleep(5)


# Load required Windows libraries
user32 = ctypes.windll.user32
imm32 = ctypes.windll.imm32

# Define the conversion mode constant for Korean native (Hangul) mode.
IME_CMODE_NATIVE = 0x0001  # When set, the IME is in Korean (Hangul) mode.

def toggle_korean_english():
    # Get the handle of the foreground window.
    hwnd = user32.GetForegroundWindow()
    if not hwnd:
        print("No active window found.")
        return

    # Get the IME context of the foreground window.
    himc = imm32.ImmGetContext(hwnd)
    if not himc:
        print("Unable to retrieve IME context.")
        return

    # Prepare DWORDs to receive the conversion and sentence modes.
    conversion = wintypes.DWORD()
    sentence = wintypes.DWORD()
    
    # Retrieve the current conversion status.
    if not imm32.ImmGetConversionStatus(himc, ctypes.byref(conversion), ctypes.byref(sentence)):
        print("Unable to get conversion status.")
        imm32.ImmReleaseContext(hwnd, himc)
        return

    current_conv = conversion.value

    # Check the current mode and toggle.
    if current_conv & IME_CMODE_NATIVE:
        # If in Korean mode, disable the native input flag for English.
        new_conv = current_conv & ~IME_CMODE_NATIVE
        new_mode = "English"
    else:
        # If in English mode, enable the native input flag for Korean.
        new_conv = current_conv | IME_CMODE_NATIVE
        new_mode = "Korean"

    # Apply the new conversion status.
    if not imm32.ImmSetConversionStatus(himc, new_conv, sentence.value):
        print("Failed to set conversion status.")
    else:
        print(f"Toggled input mode to {new_mode}.")

    # Release the IME context.
    imm32.ImmReleaseContext(hwnd, himc)

if __name__ == "__main__":
    toggle_korean_english()


5초 후 실행됩니다.
Unable to retrieve IME context.


In [ ]:
import ctypes
from ctypes import wintypes
import tkinter as tk
import keyboard  # pip install keyboard

# Constants for IME conversion mode
IME_CMODE_HANGUL = 0x0001

# Load Windows libraries
user32 = ctypes.windll.user32
imm32 = ctypes.windll.imm32

def get_ime_state(hwnd):
    """
    Retrieves the current IME conversion state for the given window handle.
    Returns "KR" if Hangul (Korean) is active, "EN" otherwise.
    """
    hIMC = imm32.ImmGetContext(hwnd)
    if not hIMC:
        return "N/A"
    conversion = wintypes.DWORD()
    sentence = wintypes.DWORD()
    if not imm32.ImmGetConversionStatus(hIMC, ctypes.byref(conversion), ctypes.byref(sentence)):
        imm32.ImmReleaseContext(hwnd, hIMC)
        return "Err"
    state = "KR" if (conversion.value & IME_CMODE_HANGUL) else "EN"
    imm32.ImmReleaseContext(hwnd, hIMC)
    return state

def toggle_ime(hwnd):
    """
    Toggles the IME conversion state for the given window handle.
    If Hangul mode is active, it switches to English, and vice versa.
    """
    hIMC = imm32.ImmGetContext(hwnd)
    if not hIMC:
        print("Unable to get the input context for toggling.")
        return
    conversion = wintypes.DWORD()
    sentence = wintypes.DWORD()
    if not imm32.ImmGetConversionStatus(hIMC, ctypes.byref(conversion), ctypes.byref(sentence)):
        imm32.ImmReleaseContext(hwnd, hIMC)
        print("Failed to get conversion status for toggling.")
        return

    # Toggle: if Hangul flag is set, remove it; if not, add it.
    if conversion.value & IME_CMODE_HANGUL:
        new_conversion = conversion.value & ~IME_CMODE_HANGUL
    else:
        new_conversion = conversion.value | IME_CMODE_HANGUL

    if not imm32.ImmSetConversionStatus(hIMC, new_conversion, sentence.value):
        print("Failed to set conversion status.")
    else:
        print("Toggled IME mode.")
    imm32.ImmReleaseContext(hwnd, hIMC)

# --- Create a small, borderless Tkinter window ---

root = tk.Tk()
root.title("IME Mode")
# Remove window borders (makes it look like a small floating widget)
root.overrideredirect(True)
# Always on top
root.attributes("-topmost", True)

# Set window size
window_width = 150
window_height = 50

# Calculate position for bottom right corner (adjust margins as needed)
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_pos = screen_width - window_width - 10   # 10-pixel margin from right edge
y_pos = screen_height - window_height - 40   # 40-pixel margin from bottom (account for taskbar)
root.geometry(f"{window_width}x{window_height}+{x_pos}+{y_pos}")

# Create a label to display the current IME state ("KR" or "EN")
state_label = tk.Label(root, text="Loading...", font=("Arial", 16), bg="black", fg="white")
state_label.pack(expand=True, fill="both")

def update_label():
    """
    Periodically checks the current IME state and updates the label.
    """
    state = get_ime_state(root.winfo_id())
    state_label.config(text=state)
    root.after(100, update_label)  # check every 100 milliseconds

update_label()  # start periodic update

def on_toggle():
    """
    Toggles the IME state when 't' is pressed, then updates the label.
    """
    toggle_ime(root.winfo_id())
    state = get_ime_state(root.winfo_id())
    state_label.config(text=state)

# Register a global hotkey for "t" to toggle IME mode.
keyboard.add_hotkey("t", on_toggle)

# Optionally, update the label on every key press (global hook)
def on_key_event(event):
    # This callback updates the label after every key press.
    state = get_ime_state(root.winfo_id())
    state_label.config(text=state)

keyboard.on_press(on_key_event)

# Start the Tkinter main loop
root.mainloop()


In [ ]:
import subprocess
import sys
import platform
from pynput import keyboard
from datetime import datetime
import time
import ctypes  # Needed for Windows API calls
from langdetect import detect, LangDetectException
from ctypes import wintypes


# ULONG_PTR 타입 정의
wintypes.ULONG_PTR = wintypes.WPARAM

# User32.dll 로드
hllDll = ctypes.WinDLL ("User32.dll", use_last_error=True)

# 한/영 키 Virtual Key 코드 (VK_HANGUEL)
VK_HANGUEL = 0x15

# INPUT 구조체 정의
class MOUSEINPUT(ctypes.Structure):
    _fields_ = (("dx",          wintypes.LONG),
                ("dy",          wintypes.LONG),
                ("mouseData",   wintypes.DWORD),
                ("dwFlags",     wintypes.DWORD),
                ("time",        wintypes.DWORD),
                ("dwExtraInfo", wintypes.ULONG_PTR))

class HARDWAREINPUT(ctypes.Structure):
    _fields_ = (("uMsg",    wintypes.DWORD),
                ("wParamL", wintypes.WORD),
                ("wParamH", wintypes.WORD))

class KEYBDINPUT(ctypes.Structure):
    _fields_ = (("wVk",         wintypes.WORD), # Virtual-key code
                ("wScan",       wintypes.WORD), # Hardware scan code
                ("dwFlags",     wintypes.DWORD),# 플래그
                ("time",        wintypes.DWORD),# 시간
                ("dwExtraInfo", wintypes.ULONG_PTR))# 추가 정보

class INPUT(ctypes.Structure):
    class _INPUT(ctypes.Union):
        _fields_ = (("ki", KEYBDINPUT),
                    ("mi", MOUSEINPUT),
                    ("hi", HARDWAREINPUT))
    _anonymous_ = ("_input",) # 익명 유니온
    _fields_ = (("type",   wintypes.DWORD), # 입력 타입
                ("_input", _INPUT))       # 유니온 구조체

# 한/영 상태 가져오는 함수
def get_hanguel_state():
    return hllDll.GetKeyState(VK_HANGUEL)

# 한/영 상태 변경 함수
def change_state():
    x = INPUT(type=1 ,ki=KEYBDINPUT(wVk=VK_HANGUEL)) # 키 누름 이벤트
    y = INPUT(type=1, ki=KEYBDINPUT(wVk=VK_HANGUEL,dwFlags=2)) # 키 뗌 이벤트
    hllDll.SendInput(1, ctypes.byref(x), ctypes.sizeof(x)) # 키 누름 이벤트 전송
    time.sleep(0.05) # 딜레이
    hllDll.SendInput(1, ctypes.byref(y), ctypes.sizeof(y)) # 키 뗌 이벤트 전송

# 현재 언어 모드 확인 함수 (영어 또는 한글)
def check_language_mode():
    hanguel_state = get_hanguel_state() # 한/영 키 상태 가져오기
    if hanguel_state == 0:
        return "English" # 0이면 영어 모드 (한글 키 비활성화)
    elif hanguel_state == 1:
        return "Korean" # 1이면 한글 모드 (한글 키 활성화)
    else:
        return "Unknown" # 0, 1 외의 경우 (혹시 모를 오류 대비)


for i in range(5):
    time.sleep(5)
    print(check_language_mode())
    

    

Unknown
Unknown
